In [1]:
import torch
import torch.nn as nn
import tensorrt as trt
from torch2trt.torch2trt import *
from torch2trt.module_test import add_module_test
# from TCN.tcn_inference import TensorCache

In [2]:
class TensorCacheTRT(nn.Module):
    def __init__(self, tensor):
        super(TensorCacheTRT, self).__init__()
        self.register_buffer('cache', tensor)
        # self.cache = tensor
    
    def forward(self, x):
        # assert x.size() == self.cache[:,:,0:1].size()
        cache_update = torch.cat((self.cache[:,1:], x.detach()), dim=1)
        self.cache[:,:] = cache_update
        return self.cache

In [76]:
@tensorrt_converter(TensorCacheTRT.forward)
def convert_TensorCache(ctx):
    module = ctx.method_args[0]
#     module.cache._trt = trt.ITensor(ctx.method_return[0])
    print(module.cache)
    output = ctx.method_return[0]
    print(output)
    input_tensor = ctx.method_args[1]
    print(input_tensor)
    layer = ctx.network.add_constant(tuple(output.shape), output.detach().cpu().numpy())
    print(layer.get_output(0))
    output._trt = layer.get_output(0)
    module.cache._trt = layer.get_output(0)
    
    # print(output)
    # layer = ctx.network.add_constant(tuple(output.shape), output.detach().cpu().numpy() )
    # output._trt = layer.get_output(0)



In [3]:
tc = TensorCacheTRT(torch.zeros(1,10).cuda())

In [4]:
tc_trt = torch2trt(tc, torch.tensor([[[0.]]]).cuda())

AttributeError: 'Tensor' object has no attribute '_trt'

In [67]:
tc(torch.tensor([[2.]]).cuda())

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 2.]], device='cuda:0')

In [68]:
tc_trt(torch.tensor([[2.]]).cuda())

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], device='cuda:0')

In [11]:
tt(torch.tensor([(1., 2., 3.)]).cuda())

tensor([[2., 4., 6.],
        [5., 7., 9.]], device='cuda:0')

AttributeError: module 'torch2trt' has no attribute '__version__'